In [27]:
# Data prep
import os
import pandas as pd
import numpy as  np
import re
import warnings
from datetime import date, timedelta
warnings.filterwarnings('ignore')

### Users

In [2]:
dfUsers = pd.read_csv('raw/users.csv')
dfUsers

,id,name,address,postalcode,city,birthday,phone,country,active,excluded
0,1,Breena Connow,16 Acker Alley,NaN,Villa Serrano,31/12/1978,722-967-3477,Bolivia,True,False
1,2,Harris Dunkerton,8356 Talisman Crossing,99871,Inari,12/03/1990,909-642-7516,Finland,True,True
2,3,Merrie Grindle,3 Homewood Junction,NaN,Cengjia,13/10/1944,876-111-3766,China,False,True
3,4,Arlan Morstatt,05 Straubel Place,13315-000,Cabreúva,03/03/1945,870-897-0217,Brazil,True,False
4,5,Kiele Dedrick,947 Mallard Drive,3515-517,Bodiosa a Velha,26/02/1950,879-743-0896,Portugal,True,False
...,...,...,...,...,...,...,...,...,...,...
995,996,Jere Matys,34 Park Meadow Crossing,195047,Florencia,01/06/1960,967-680-6203,Colombia,True,False
996,997,Bax Levick,17468 Kropf Court,NaN,Gaotang,21/12/1961,399-742-6576,China,True,False
997,998,Hermine Bowerbank,4100 Brown Avenue,32-060,Rzozów,12/06/1989,240-137-7616,Poland,False,True
998,999,Beaufort Caulier,72724 Rigney Hill,11320,Kandana,13/02/1999,961-971-2453,Sri Lanka,False,True


In [4]:
dfUsers.loc[dfUsers["active"] == True, "excluded"] = False
dfUsers['excluded'].value_counts()

False    744
True     256
Name: excluded, dtype: int64

In [7]:
dfUsers['birthday'] = pd.to_datetime(dfUsers['birthday'])
dfUsers['age'] = 2022 - dfUsers['birthday'].dt.year
dfUsers.dtypes

id                     int64
name                  object
address               object
postalcode            object
city                  object
birthday      datetime64[ns]
phone                 object
country               object
active                  bool
excluded                bool
age                    int64
dtype: object

In [8]:
dfUsers['name'] = dfUsers['name'].apply(lambda x: x.replace("'", " "))
dfUsers['city'] = dfUsers['city'].apply(lambda x: x.replace("'", " "))
dfUsers

,id,name,address,postalcode,city,birthday,phone,country,active,excluded,age
0,1,Breena Connow,16 Acker Alley,NaN,Villa Serrano,1978-12-31,722-967-3477,Bolivia,True,False,44
1,2,Harris Dunkerton,8356 Talisman Crossing,99871,Inari,1990-12-03,909-642-7516,Finland,True,False,32
2,3,Merrie Grindle,3 Homewood Junction,NaN,Cengjia,1944-10-13,876-111-3766,China,False,True,78
3,4,Arlan Morstatt,05 Straubel Place,13315-000,Cabreúva,1945-03-03,870-897-0217,Brazil,True,False,77
4,5,Kiele Dedrick,947 Mallard Drive,3515-517,Bodiosa a Velha,1950-02-26,879-743-0896,Portugal,True,False,72
...,...,...,...,...,...,...,...,...,...,...,...
995,996,Jere Matys,34 Park Meadow Crossing,195047,Florencia,1960-01-06,967-680-6203,Colombia,True,False,62
996,997,Bax Levick,17468 Kropf Court,NaN,Gaotang,1961-12-21,399-742-6576,China,True,False,61
997,998,Hermine Bowerbank,4100 Brown Avenue,32-060,Rzozów,1989-12-06,240-137-7616,Poland,False,True,33
998,999,Beaufort Caulier,72724 Rigney Hill,11320,Kandana,1999-02-13,961-971-2453,Sri Lanka,False,True,23


In [9]:
dfUsers.to_csv('prepared/users.csv')

### Events

#### Football

In [136]:
def processFootball(filename):
    df = pd.read_csv('raw/' + filename + '.csv')
    df = df[df['Home'].notna()]
    df['Date'] = pd.to_datetime(df['Date'])
    df.sort_values(by='Date')
    df['Name'] = df['Home'] + ' VS ' + df['Away']
    df['Name'] = df['Name'].apply(lambda x: x.replace("'", " "))
    df['Name'] = df['Name'].apply(lambda x: x.replace("-", " "))
    df = df.drop(columns=['Round', 'Wk', 'Day', 'Time', 'Attendance', 'Venue', 'Referee', 'Match Report', 'Notes', 'Home', 'Away', 'xG', 'xG.1'], errors='ignore')
    df.insert(0, 'Category', 'Football')
    twodaysmarging = pd.to_datetime(date.today() - timedelta(2))
    #print(twodaysmarging)
    df['Active'] = np.where(df['Date'] > twodaysmarging, True, False)
    df.loc[df["Score"] == np.NaN, "Active"] = True
    #df = df[df.Score != 'nan']
    return df

In [137]:
pt = processFootball('primeiraliga')
de = processFootball('bundesliga')
en = processFootball('epl')
es = processFootball('laliga')
fr = processFootball('ligue1')
it = processFootball('seriea')
wc = processFootball('fifaworldcup')
cl = processFootball('championsleague')
el = processFootball('europaleague')

In [138]:
football = pd.concat([pt, de, en, es, fr, it, wc, cl, el])
football.sort_values(by='Date')
fourmonthsspan = pd.to_datetime(date.today() + timedelta(121))
football = football[(football['Date'] < fourmonthsspan)]
football['Name'] = football['Name'].apply(lambda x: re.sub('[a-z]{2,3}\sVS\s[a-z]{2,3}', 'VS', x))
football.insert(0, 'ID', range(0, 0 + len(football)))
football

,ID,Category,Date,Score,Name,Active
0,0,Football,2022-08-05,4–0,Benfica VS Arouca,False
1,1,Football,2022-08-06,0–1,Rio Ave VS Vizela,False
2,2,Football,2022-08-06,2–0,Estoril VS Famalicão,False
3,3,Football,2022-08-06,5–1,Porto VS Marítimo,False
4,4,Football,2022-08-07,0–0,Santa Clara VS Casa Pia,False
...,...,...,...,...,...,...
114,1905,Football,2023-02-23,NaN,Nantes VS Juventus,True
115,1906,Football,2023-02-23,NaN,Manchester Utd VS Barcelona,True
116,1907,Football,2023-02-23,NaN,Roma VS RB Salzburg,True
117,1908,Football,2023-02-23,NaN,Rennes VS Shakhtar,True


In [139]:
football.to_csv('prepared/football.csv')

#### Basketball

In [124]:
twodaysmarging = pd.to_datetime(date.today() - timedelta(2))

nba = pd.read_csv('raw/basketballnba.csv')

scores = []
for index, row in nba.iterrows():
    if type(row['PTS-V']) == float and pd.notna(row['PTS-V']):
        scores.append(str(int(row['PTS-H'])) + ' - ' + str(int(row['PTS-V'])))
    else:
        scores.append(np.nan)

nba['Score'] = scores

nba = nba[nba['Home/Neutral'].notna()]
nba['Date'] = pd.to_datetime(nba['Date'])
nba = nba[(nba['Date'] < fourmonthsspan)]
nba['Name'] = nba['Home/Neutral'] + ' - ' + nba['Visitor/Neutral']
nba = nba.drop(columns=['Start (ET)', 'Visitor/Neutral', 'PTS-V', 'Home/Neutral', 'PTS-H', 'Unnamed: 6', 'Unnamed: 7', 'Attend.', 'Arena', 'Notes'])
nba.insert(0, 'Category', 'Basketball')
nba.insert(0, 'ID', range(0, 0 + len(nba)))
nba['Active'] = np.where(nba['Date'] > twodaysmarging, True, False)
nba

,ID,Category,Date,Score,Name,Active
0,0,Basketball,2022-10-18,126 - 117,Boston Celtics - Philadelphia 76ers,False
1,1,Basketball,2022-10-18,123 - 109,Golden State Warriors - Los Angeles Lakers,False
2,2,Basketball,2022-10-19,113 - 109,Detroit Pistons - Orlando Magic,False
3,3,Basketball,2022-10-19,107 - 114,Indiana Pacers - Washington Wizards,False
4,4,Basketball,2022-10-19,117 - 107,Atlanta Hawks - Houston Rockets,False
...,...,...,...,...,...,...
1210,1210,Basketball,2023-04-07,NaN,Sacramento Kings - Golden State Warriors,True
1211,1211,Basketball,2023-04-07,NaN,Los Angeles Lakers - Phoenix Suns,True
1212,1212,Basketball,2023-04-08,NaN,Utah Jazz - Denver Nuggets,True
1213,1213,Basketball,2023-04-08,NaN,Los Angeles Clippers - Portland Trail Blazers,True


In [125]:
nba.to_csv('prepared/basketball.csv')

#### American Football

In [126]:
nfl = pd.read_csv('raw/americanfootballnfl.csv')
nfl = nfl[nfl['Winner/tie'].notna()]

scores = []
for index, row in nfl.iterrows():
    if type(row['PtsW']) == float and pd.notna(row['PtsW']):
        scores.append(str(int(row['PtsW'])) + ' - ' + str(int(row['PtsL'])))
    else:
        scores.append(np.nan)
nfl['Score'] = scores

nfl['Date'] = pd.to_datetime(nfl['Date'])
nfl = nfl[(nfl['Date'] < fourmonthsspan)]
nfl['Name'] = nfl['Winner/tie'] + ' - ' + nfl['Loser/tie']
nfl = nfl.drop(columns=['Week', 'Day', 'Time', 'Winner/tie', 'Unnamed: 5', 'Loser/tie', 'Unnamed: 7', 'PtsW', 'PtsL', 'YdsW', 'TOW', 'YdsL', 'TOL'])
nfl.insert(0, 'Category', 'American Football')
nfl.insert(0, 'ID', range(0, 0 + len(nfl)))
nfl['Active'] = np.where(nfl['Date'] > twodaysmarging, True, False)
nfl

,ID,Category,Date,Score,Name,Active
0,0,American Football,2022-09-08,31 - 10,Buffalo Bills - Los Angeles Rams,False
1,1,American Football,2022-09-11,27 - 26,New Orleans Saints - Atlanta Falcons,False
2,2,American Football,2022-09-11,26 - 24,Cleveland Browns - Carolina Panthers,False
3,3,American Football,2022-09-11,19 - 10,Chicago Bears - San Francisco 49ers,False
4,4,American Football,2022-09-11,23 - 20,Pittsburgh Steelers - Cincinnati Bengals,False
...,...,...,...,...,...,...
267,267,American Football,2023-01-08,NaN,New England Patriots - Buffalo Bills,True
268,268,American Football,2023-01-08,NaN,Houston Texans - Indianapolis Colts,True
269,269,American Football,2023-01-08,NaN,Kansas City Chiefs - Las Vegas Raiders,True
270,270,American Football,2023-01-08,NaN,Tampa Bay Buccaneers - Atlanta Falcons,True


In [127]:
nfl.to_csv('prepared/americanfootball.csv')

#### Hockey

In [129]:
nhl = pd.read_csv('raw/hockeynhl.csv')
nhl = nhl[nhl['Home'].notna()]
nhl['Date'] = pd.to_datetime(nhl['Date'])
nhl['Name'] = nhl['Home'] + ' - ' + nhl['Visitor']

scores = []
for index, row in nhl.iterrows():
    if type(row['G']) == float and pd.notna(row['G']):
        scores.append(str(int(row['G.1'])) + ' - ' + str(int(row['G'])))
    else:
        scores.append(np.nan)
nhl['Score'] = scores

nhl = nhl[(nhl['Date'] < fourmonthsspan)]
nhl = nhl.drop(columns=['Visitor', 'G', 'Home', 'G.1', 'Unnamed: 5', 'Att.', 'LOG', 'Notes'])
nhl.insert(0, 'Category', 'Hockey')
nhl.insert(0, 'ID', range(0, 0 + len(nhl)))
nhl['Active'] = np.where(nhl['Date'] > twodaysmarging, True, False)
nhl

,ID,Category,Date,Name,Score,Active
0,0,Hockey,2022-10-07,Nashville Predators - San Jose Sharks,4 - 1,False
1,1,Hockey,2022-10-08,San Jose Sharks - Nashville Predators,2 - 3,False
2,2,Hockey,2022-10-11,Los Angeles Kings - Vegas Golden Knights,3 - 4,False
3,3,Hockey,2022-10-11,New York Rangers - Tampa Bay Lightning,3 - 1,False
4,4,Hockey,2022-10-12,Anaheim Ducks - Seattle Kraken,5 - 4,False
...,...,...,...,...,...,...
1266,1266,Hockey,2023-04-08,San Jose Sharks - Edmonton Oilers,NaN,True
1267,1267,Hockey,2023-04-08,Toronto Maple Leafs - Montreal Canadiens,NaN,True
1268,1268,Hockey,2023-04-08,Vancouver Canucks - Calgary Flames,NaN,True
1269,1269,Hockey,2023-04-08,Winnipeg Jets - Nashville Predators,NaN,True


In [130]:
nhl.to_csv('prepared/hockey.csv')